## **利用ogr开源库进行矢量文件的读写**


In [25]:
from osgeo import ogr


#### **1. 用python读入矢量文件**

In [26]:
path_kunming = 'data/昆明市行政边界(poi86.com)/昆明市边界_wgs84.shp'
path_kunming_out_1 = 'data/Section_4/vector_out_1.shp'
path_kunming_out_2 = 'data/Section_4/vector_out_2.shp'


In [27]:
### 读入矢量文件
ds = ogr.Open(path_kunming, 0) #0是只读，1是可写
layer = ds.GetLayer(0)  # 获取矢量文件Layer
num_fea = layer.GetFeatureCount()   ## 统计要素个数
print('Number of feature: ', num_fea)
num_field = layer.GetFeature(0).GetFieldCount()   ## 统计字数个数
print('Number of fields:', num_field)


Number of feature:  15
Number of fields: 3


#### **2.查看矢量文件字段**

In [28]:
### 查看字段名
fields = []
fea = layer.GetFeature(0)
for i in range(0, num_field):
    field = fea.GetFieldDefnRef(i).name
    fields.append(field)
print(fields)


['fid', 'name', 'site']


In [29]:
### 查看字段值
field_name = 'name'
fea = layer.GetFeature(1)
print('Filed value: ',fea.GetField(field_name))
### 获得所有要素的字段值
for fea in layer:   ## 
  print(fea.GetField(field_name))


Filed value:  五华区
昆明市
五华区
盘龙区
官渡区
西山区
东川区
呈贡区
晋宁区
富民县
宜良县
石林彝族自治县
嵩明县
禄劝彝族苗族自治县
寻甸回族彝族自治县
安宁市


#### **3. 写出矢量文件**

#### 3.1 写出只含有特定字段的矢量文件

In [30]:
field_name = 'fid'
ds = ogr.Open(path_kunming, 0)  # 0是只读，1是可写
layer = ds.GetLayer()    # 获取矢量文件图层Layer
driver = ogr.GetDriverByName('ESRI Shapefile')   ## 文件驱动（用于写出文件）
ds_out = driver.CreateDataSource(path_kunming_out_1)  ## 创建数据源DataSource
layer_out = ds_out.CreateLayer('vector_new_1', geom_type=ogr.wkbPolygon, srs=layer.GetSpatialRef())
field_defn = ogr.FieldDefn(field_name, ogr.OFTReal)   ## 定义字段
layer_out.CreateField(field_defn)    ## 新建字段
# fea_defn = layer_out.GetFeature(0).GetDefnRef()    ## layer_out没有要素，故.GetFeature(0)会报错
fea_defn = layer_out.GetLayerDefn()   ## 获得特征定义，用于特征
### 创建要素（将复制‘昆明市边界_wgs84.shp’文件中要素)
for fea in layer:
    geo = fea.geometry()         ## 获得原矢量要素的几何
    fea_out = ogr.Feature(fea_defn)
    fea_out.SetGeometry(geo)
    fea_out.SetField(field_name, fea.GetField(field_name))
    layer_out.CreateFeature(fea_out)
ds_out = None       #  ### 保存/关闭 DataSource (!重要)


#### 3.2 写出只含有呈贡区的矢量文件


In [31]:
ds = ogr.Open(path_kunming, 0) # 0是只读，1是可写
layer = ds.GetLayer('昆明市边界_wgs84')   # 获取矢量文件Layer
driver = ogr.GetDriverByName('ESRI Shapefile')
ds_out = driver.CreateDataSource(path_kunming_out_2)
layer_out = ds_out.CreateLayer('vector_new_2', geom_type=ogr.wkbPolygon, srs=layer.GetSpatialRef())
fea_defn = layer_out.GetLayerDefn()   ## 获得空要素定义(或空图层定义)

### 创建字段
layer_defn = layer_out.GetLayerDefn()
print('Number of fields (before field creating): ', layer_defn.GetFieldCount())  ### 字段创建前字段数
fea = layer.GetFeature(0)
for i in range(fea.GetFieldCount()):
  field_defn = fea.GetFieldDefnRef(i)
  layer_out.CreateField(field_defn)
layer_defn = layer_out.GetLayerDefn()
print('Number of fields (after field creating): ', layer_defn.GetFieldCount())

### 创建要素（复制‘昆明市边界_wgs84.shp’文件中要素)
for fea in layer:
  if fea.GetField('name') == '呈贡区':
    fea_out = ogr.Feature(fea_defn)
    fea_out.SetGeometry(fea.geometry())
    for i in range(fea.GetFieldCount()):
      field_value = fea.GetField(i)
      fea_out.SetField(i, field_value)
    layer_out.CreateFeature(fea_out)
ds_out = None        #  ### 保存/关闭 DataSource (重要)



Number of fields (before field creating):  0
Number of fields (after field creating):  3


Warning 1: One or several characters couldn't be converted correctly from UTF-8 to ISO-8859-1.  This warning will not be emitted anymore.


### 作业：写出只含五华区、盘龙区、西山区、官渡区、呈贡区的矢量文件。